<a href="https://colab.research.google.com/github/Harisudhan23/NLP/blob/main/NLP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
! pip install transformers torch sklearn sacrebleu gensim
!pip install rouge_score
!pip install sentencepiece
!pip install sacrebleu

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.7 MB/s eta 0:00:00


In [9]:
import numpy as np
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, pipeline
from torch.utils.data import Dataset
from rouge_score import rouge_scorer
from sacrebleu import sentence_bleu
import gensim.downloader as api


In [10]:
# Sample dataset
texts = [
    "I love programming in Python.",
    "The movie was a terrible waste of time.",
    "Artificial Intelligence is shaping the future.",
    "This book was boring and not worth reading.",
    "AI is making the world smarter and more efficient.",
    "I didn't enjoy the movie at all."
]
labels = [1, 0, 1, 0, 1, 0]  # 1: Positive, 0: Negative


In [11]:
# Bag-of-Words Vectorization
bow_vectorizer = CountVectorizer()
X_bow = bow_vectorizer.fit_transform(texts)
X_train_bow, X_test_bow, y_train, y_test = train_test_split(X_bow, labels, test_size=0.3, random_state=42)

# Train SVM Classifier
bow_model = SVC(kernel="linear")
bow_model.fit(X_train_bow, y_train)
pred_bow = bow_model.predict(X_test_bow)

print("BoW Accuracy:", accuracy_score(y_test, pred_bow))
print("Classification Report (BoW):\n", classification_report(y_test, pred_bow))


BoW Accuracy: 0.5
Classification Report (BoW):
               precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
# Load GloVe embeddings
glove_vectors = api.load("glove-wiki-gigaword-50")

# Convert text to GloVe embeddings
def text_to_glove_vector(text):
    words = text.split()
    vectors = [glove_vectors[word] for word in words if word in glove_vectors]
    return np.mean(vectors, axis=0) if vectors else np.zeros(50)

X_glove = np.array([text_to_glove_vector(text) for text in texts])
X_train_glove, X_test_glove, _, _ = train_test_split(X_glove, labels, test_size=0.3, random_state=42)

# Train SVM Classifier
glove_model = SVC(kernel="linear")
glove_model.fit(X_train_glove, y_train)
pred_glove = glove_model.predict(X_test_glove)

print("GloVe Accuracy:", accuracy_score(y_test, pred_glove))
print("Classification Report (GloVe):\n", classification_report(y_test, pred_glove))


[==================================================] 100.0% 66.0/66.0MB downloaded
GloVe Accuracy: 0.5
Classification Report (GloVe):
               precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label),
        }


In [14]:
# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Prepare Dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.3, random_state=42)
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    evaluation_strategy="epoch",
    logging_dir="./logs",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train
trainer.train()

# Predict
predictions = trainer.predict(test_dataset)
y_pred_bert = np.argmax(predictions.predictions, axis=-1)

print("BERT Accuracy:", accuracy_score(test_labels, y_pred_bert))
print("Classification Report (BERT):\n", classification_report(test_labels, y_pred_bert))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.566090
2,No log,0.558301


BERT Accuracy: 1.0
Classification Report (BERT):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [15]:
# Summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = """
Artificial Intelligence (AI) is transforming industries by automating tasks, improving decision-making,
and enabling innovative solutions to complex problems. From healthcare to finance, AI's impact is profound.
"""

summary = summarizer(text, max_length=50, min_length=25, do_sample=False)[0]['summary_text']
reference_summary = "AI is transforming industries by automating tasks and enabling innovation."

# ROUGE Scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, summary)
print("ROUGE Scores:", scores)

# BLEU Score
bleu_score = sentence_bleu(summary, [reference_summary])
print("BLEU Score:", bleu_score.score)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Your max_length is set to 50, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)


ROUGE Scores: {'rouge1': Score(precision=0.4, recall=1.0, fmeasure=0.5714285714285715), 'rougeL': Score(precision=0.4, recall=1.0, fmeasure=0.5714285714285715)}
BLEU Score: 18.189587992135596
